# Імпорт бібілотек

In [ ]:

# !pip install pandas
# !pip install numpy
# !pip install tensorflow
# !pip install setuptools
# !pip install keras
# !pip install keras-tuner
# !pip install seaborn
# !pip install matplotlib
# !pip install scikit-learn
# !pip install scipy
# !pip install statsmodels
# !pip install pvlib
# !pip install xgboost
# !pip install ipython

In [ ]:
import math
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import LSTM, Dropout, BatchNormalization, Dense, Input, Conv1D, MaxPooling1D, Reshape, GRU, Bidirectional
from keras.layers import MultiHeadAttention, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.regularizers import l2
from sklearn.metrics import r2_score, explained_variance_score, mean_squared_log_error, mean_squared_error, mean_absolute_percentage_error, mean_absolute_error

from scipy.stats import pearsonr
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from statsmodels.graphics.tsaplots import plot_acf
from sklearn.metrics import mean_absolute_percentage_error

from pandas.plotting import scatter_matrix
from IPython.display import display
import pvlib

import logging
logger = logging.getLogger(__name__)


# Завантаження даних

In [ ]:
data_name_predict = 'Glyniany-1_OpenMeteo_Data'
data_name_actual = 'Glyniany-1_Nordik_Data'
power_max = 7153 # максимальна потужність станції
latitude = 49.308290
longitude = 23.430082

# data_name = 'Glyniany-1_OpenMeteo_Data'
# power_max = 2976
# latitude = 49.826009
# longitude = 24.476087

# data_name = 'Glyniany-2_OpenMeteo_Data'
# power_max = 15442
# latitude = 49.826411
# longitude = 24.488083

# data_name = 'Yavoriv_OpenMeteo_Data'
# power_max = 59234
# latitude = 49.935239
# longitude = 23.505527

data_period = 15 # періодичність даних

drop_invalid_values = True # видалення недійсних значень

datetime_start = pd.to_datetime('2024-09-21 00:00:00')
datetime_end = pd.to_datetime('2024-11-24 23:59:59')

hour_start = 0
hour_end = 23

time_steps = 1
train_percent = 0.2
power_min_limit = 0.01

model_tunning = False

MODEL_DATA_PATH = 'model_data'

models_metrics = { }

In [ ]:
# Завантаження даних
df_predict = pd.read_csv(f'../Data/{data_name_predict}.csv', sep=',', comment='#', low_memory=False)
df_predict = df_predict.rename(columns={col: f"{col}_predict" for col in df_predict.columns if col not in ['datetime', 'power_actual', 'limitation_actual']})

df_actual = pd.read_csv(f'../Data/{data_name_actual}.csv', sep=',', comment='#', low_memory=False)
df_actual = df_actual.rename(columns={col: f"{col}_actual" for col in df_actual.columns if col not in ['datetime', 'power_actual', 'limitation_actual']})

df = pd.DataFrame()
df = pd.merge(df_actual, df_predict, on='datetime', how='outer', suffixes=('_actual', '_predict'))
df['power_predict'] = 0
df.drop(['limitation_predict'], inplace=True, axis=1)

df.columns

In [ ]:
df

# Статистична інформація

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
df.isnull().sum()

**Опис даних**


| Поле                      | Опис                                                                                     |
|---------------------------|-------------------------------------------------------------------------------------------|
| `datetime`                | Дата та час вимірювань. Періодичність 15 хв.                                             |
| `power_actual`            | Фактична згенерована потужність за період. Діапазон від -32 до 12895.                     |
| `limitation_actual`       | Фактичні обмеження вироблення енергії. Категорійна змінна. 1 - обмеження чи несправності. |
| `ghi_actual`              | Фактичне загальне горизонтальне випромінювання. Діапазон від -10 до 894.                 |
| `gti_actual`              | Фактичне загальне випромінювання на нахилену поверхню. Діапазон: від 0 до 1051.          |
| `temperature_pv_actual`   | Фактична температура сонячних панелей в градусах Цельсія.                                 |
| `cloud_cover_high`        | Прогнозована хмарність на великій висоті. Діапазон: від 0% до 100%.                      |
| `cloud_cover_mid`         | Прогнозована хмарність на середній висоті. Діапазон: від 0% до 100%.                     |
| `cloud_cover_low`         | Прогнозована хмарність на низькій висоті. Діапазон: від 0% до 100%.                      |
| `dew_point_predict`       | Прогнозована точка роси. Діапазон від -18.58°C до 21.61°C. Середнє значення: 6.47°C.     |
| `dni_predict`             | Прогнозоване пряме нормальне випромінювання. Діапазон: від 0 до 832.                     |
| `ghi_predict`             | Прогнозоване загальне горизонтальне випромінювання. Діапазон від -10 до 894.             |
| `gti_predict`             | Прогнозоване загальне випромінювання на нахилену поверхню. Діапазон: від 0 до 1051.      |
| `humidity_predict`        | Прогнозована відносна вологість. Діапазон: від 24% до 100%.                              |
| `pressure_air_predict`    | Прогнозований атмосферний тиск у гПа. Діапазон: від 946.85 до 995.18.                    |
| `sunshine_predict`        | Прогнозована тривалість сонячного освітлення. Діапазон: від 0 до 900 хвилин.             |
| `temperature_air_predict` | Прогнозована температура повітря в градусах Цельсія. Діапазон: від -17.85°C до 34.5°C.   |
| `temperature_soil_predict`| Прогнозована температура ґрунту в градусах Цельсія. Діапазон: від -5.75°C до 32.65°C.    |
| `visibility_predict`      | Прогнозована видимість у метрах. Діапазон: від 20 м до 50,000 м.                         |
| `wind_direction_predict`  | Прогнозований напрямок вітру у градусах. Діапазон: від 0° до 360°.                       |
| `wind_speed_predict`      | Прогнозована швидкість вітру. Діапазон: від 0 до 94.71 м/с.                              |
| `weather_code_predict`    | Код погоди ВМО. Прогнозоване значення. Категорійна змінна.                               |


Коди інтерпретації погоди ВМО (WW)
| Код          | Опис                                           |
|--------------|------------------------------------------------|
| 0            | Ясне небо                                     |
| 1, 2, 3      | Переважно ясно, хмарно з проясненнями, і повністю хмарно |
| 45, 48       | Туман і паморозевий осадковий туман           |
| 51, 53, 55   | Мряка: Легка, помірна і сильна інтенсивність  |
| 56, 57       | Замерзаюча мряка: Легка і сильна інтенсивність |
| 61, 63, 65   | Дощ: Невеликий, помірний і сильний            |
| 66, 67       | Замерзаючий дощ: Легкий і сильний             |
| 71, 73, 75   | Снігопад: Невеликий, помірний і сильний       |
| 77           | Снігові зерна                                 |
| 80, 81, 82   | Зливи дощу: Невеликі, помірні і дуже сильні   |
| 85, 86       | Снігові зливи: Невеликі і сильні              |
| 95           | Гроза: Невелика або помірна                  |
| 96, 99       | Гроза з невеликим і сильним градом           |


**Об'єм даних**

Всього записів: 29 225


**Пропущені значення**

Кількість пропущених значень: ??? (?%)



# Аналіз даних

### Розподіли змінних

In [ ]:
def plot_distribution(data, min_clip=-1, max_clip=-1, columns_per_row = 5):
    num_rows = math.ceil(len(data.columns) / columns_per_row)

    plt.figure(figsize=(13, 2 * num_rows))
    for i, col in enumerate(data.columns):
        plt.subplot(num_rows, columns_per_row, i + 1)
        filter_data = data.copy()
        min_value = data[col].min()
        max_value = data[col].max()
        delta = max_value - min_value

        if min_clip >= 0:
            filter_data = filter_data[filter_data[col] >= min_value + delta * min_clip]
        if max_clip >= 0:
            filter_data = filter_data[filter_data[col] <= max_value + delta * max_clip]

        sns.histplot(filter_data[col], bins=25, kde=True)
    plt.tight_layout()
    plt.show()

In [ ]:
df.columns

In [ ]:
# "ghi_actual",
numerical_cols = ["power_actual", "gti_actual",  "temperature_air_actual", "temperature_pv_actual",
                  "dni_predict", "gti_predict", "ghi_predict", "temperature_air_predict", "temperature_soil_predict",
                  "dew_point_predict", "wind_speed_predict", "humidity_predict", "pressure_air_predict", "visibility_predict", 
                  "cloud_cover_high_predict", "cloud_cover_mid_predict", "cloud_cover_low_predict", "sunshine_predict", "wind_direction_predict"
                ]
df_num = df[numerical_cols].clip(upper=power_max)
plot_distribution(df_num, columns_per_row=5)

In [ ]:
# розподіл при обмеженні min_clip
plot_distribution(df_num, min_clip=0.01)

In [ ]:
# обмежити генерацію 0 та power_max
df_num = df_num[df_num['power_actual'] >= power_max*0.001]
df_num.clip(upper=power_max)

# видалити незаоповнені значення
df_num = df_num.dropna()

### Аналіз викидів

In [ ]:
columns_per_row = 5
num_rows = math.ceil(len(df_num.columns) / columns_per_row)

plt.figure(figsize=(15, 2 * num_rows))
for i, col in enumerate(df_num.columns):
    plt.subplot(num_rows, columns_per_row, i + 1)
    sns.boxplot(x=df_num[col])
    plt.title(f"{col}")
plt.tight_layout()
plt.show()


**Аналіз змінних**
- `power`: Велика кількість викидів у верхньому кінці. Високі значення потужності, які зустрічаються рідко.
Ще є кілька аномалій - значення вище потужності станції.
- `cloud_cover_high`, `cloud_cover_mid`, `cloud_cover_low`: Викидів немає або їх дуже мало (розподіл значень в межах 0–100%).
- `dew_point`: Невеликі викиди у нижньому кінці (значення нижче -10°C).
- `dni`, `ghi`, `gti`: Значна кількість викидів у верхньому кінці - дуже високі показники сонячного випромінювання.
- `humidity`: Викиди у нижньому кінці (нижче 40% вологості).
- `pressure_air`: Викиди у верхньому кінці.
- `sunshine`: Викиди відсутні, значення розподілені рівномірно.
- `temperature_air`, `temperature_soil`: Невеликі викиди у нижньому кінці (температура нижче -10°C для повітря та нижче 0°C для ґрунту).
- `visibility`: Значна кількість викидів у верхньому кінці (видимість понад 30,000 м).
- `wind_direction`: Викиди практично відсутні, значення рівномірно розподілені в межах 0–360°.
- `wind_speed`: Велика кількість викидів у верхньому кінці (швидкість понад 60 м/с).
 

### Матриця кореляції

In [ ]:

df_corr_matrix = df[numerical_cols]

plt.figure(figsize=(15, 10))
sns.heatmap(df_corr_matrix.corr(method="pearson"), annot=True, cmap='coolwarm')
plt.title('Матриця кореляції (pearson)', fontsize=16)
plt.tight_layout()  
plt.show()

In [ ]:
df_corr_matrix = df[numerical_cols]

plt.figure(figsize=(15, 10))
sns.heatmap(df_corr_matrix.corr(method="spearman"), annot=True, cmap='coolwarm')
plt.title('Матриця кореляції (spearman)', fontsize=16)
plt.tight_layout()  
plt.show()

Матриця кореляції ілюструє взаємозв'язки між різними метеорологічними показниками та цільовою змінною — генерацією електроенергії (power).

Найсильніший позитивний зв’язок із цільовою змінною спостерігається для параметрів, пов’язаних із сонячним випромінюванням: DNI (0.77), GHI (0.84) і GTI (0.85), що свідчить про те, що інтенсивність сонячної радіації є ключовим фактором у процесі генерації електроенергії. Високий ступінь кореляції між цими показниками також вказує на їх сильну взаємозалежність, що свідчить про присутність мультиколінеарності. 
Температура повітря та ґрунту також мають позитивний вплив на генерацію, причому температура ґрунту (0.53) виявляється більш значущою порівняно з температурою повітря (0.42). Висока та середня хмарність (cloud_cover_high і cloud_cover_mid) негативно впливають на генерацію, тоді як низька хмарність (cloud_cover_low) має слабкіший негативний ефект. Це підтверджує, що хмарність знижує рівень сонячного випромінювання, що своєю чергою зменшує виробництво електроенергії. Вологість демонструє значну негативну кореляцію (-0.54), що пояснюється зменшенням проникності сонячної радіації при її зростанні.
Швидкість вітру та атмосферний тиск, показують слабкий або помірний вплив. Швидкість вітру має слабку негативну кореляцію (-0.09), що свідчить про її незначний вплив на ефективність роботи сонячних панелей. Атмосферний тиск має майже нейтральний вплив із незначною позитивною кореляцією (0.08).

Мультиколінеарність найбільш помітна серед параметрів, пов’язаних із сонячною радіацією. Високі кореляції між GHI, GTI та DNI можуть створювати проблеми надмірної залежності в моделі. Схожа ситуація спостерігається між температурами повітря і ґрунту (0.97), що свідчить про дублювання інформації.


### Кореляційні графіки

In [ ]:
plt.figure(figsize=(30, 30))
scatter_matrix(df_num[numerical_cols].astype(float), alpha=0.5, diagonal="kde", figsize=(30, 30))
plt.show()

Залежність між `power` та `gti` має чітко виражений тренд, але не є строго лінійною

Спостерігається нверсна залежність між `power` і `humidity`


Прослідковується слабка зворотна залежність між `temperature_air` і `pressure_air` (чіткий негативний нахил на діаграмі розсіювання)

Спостерігається сильний зв’язок із позитивним трендом між `temperature_air` і `dew_point`

Графік залежність між `gti` та `humidity` демонструє інверсну кореляцію. При низьких значеннях `gti` спостерігається широкий діапазон значень вологості `humidity`, але з переважно високою вологістю. При збільшенні `gti` вологість поступово знижується, показуючи явну негативну залежність. Ця залежність вказує на те, що більш інтенсивна сонячна радіація `gti` зазвичай супроводжується меншою вологістю


In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(df_num['power_actual'], df_num['gti_actual'], alpha=0.2, edgecolor='k')
plt.title("Power vs DTI actual", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("DTI, Wt/m²", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.subplot(1, 3, 2) 
plt.scatter(df_num['power_actual'], df_num['gti_actual'], alpha=0.2, edgecolor='k')
plt.title("Power vs GHI actual", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("GHI, Wt/m²", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(df_num['power_actual'], df_num['gti_predict'], alpha=0.2, edgecolor='k')
plt.title("Power vs DTI predict", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("DTI, Wt/m²", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.subplot(1, 3, 2) 
plt.scatter(df_num['power_actual'], df_num['ghi_predict'], alpha=0.2, edgecolor='k')
plt.title("Power vs GHI predict", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("GHI, Wt/m²", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.subplot(1, 3, 3)
plt.scatter(df_num['power_actual'], df_num['dni_predict'], alpha=0.2, edgecolor='k')
plt.title("Power vs DNI predict", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("DNI, Wt/m²", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(df_num['power_actual'], df_num['temperature_air_actual'], alpha=0.2, edgecolor='k')
plt.title("Power vs Temperature_air actual", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("Temperature_air, °C", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.subplot(1, 3, 2) 
plt.scatter(df_num['power_actual'], df_num['temperature_pv_actual'], alpha=0.2, edgecolor='k')
plt.title("Power vs Temperature_pv actual", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("Temperature_soil, °C", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(15, 5))

plt.subplot(1, 3, 1)
plt.scatter(df_num['power_actual'], df_num['temperature_air_predict'], alpha=0.2, edgecolor='k')
plt.title("Power vs Temperature_air", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("Temperature_air, °C", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.subplot(1, 3, 2) 
plt.scatter(df_num['power_actual'], df_num['temperature_soil_predict'], alpha=0.2, edgecolor='k')
plt.title("Power vs Temperature_soil", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("Temperature_soil, °C", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.subplot(1, 3, 3) 
plt.scatter(df_num['power_actual'], df_num['humidity_predict'], alpha=0.2, edgecolor='k')
plt.title("Power vs Humidity", fontsize=12)
plt.xlabel("Power, kWt", fontsize=12)
plt.ylabel("Humidity, %", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.2)

plt.tight_layout()
plt.show()

### Колеограма

In [ ]:
lags_list = [96/2, 96, 96*7]

fig, axes = plt.subplots(1, len(lags_list), figsize=(18, 6), sharey=True)

for i, lags in enumerate(lags_list):
    plot_acf(df.dropna()["power_actual"], lags=lags, ax=axes[i])
    axes[i].set_title(f"Колеограма (lags={lags})")
    axes[i].grid(True)

plt.tight_layout()
plt.show()

**Характеристики автокориляційної функції (ACF)**

- Автокореляція з лагом 1: Значення ряду на поточному кроці тісно пов’язане зі значенням попереднього кроку, що є типовою ознакою часових рядів. Початкові лаги (від 0 до 10) мають високу позитивну кореляцію.
- Тренд зменшення автокореляції: На графіку видно, як автокореляція зменшується з кожним лагом. Спад сигналу означає, що є сильна залежність між близькими часовими точками, яка поступово зникає з ростом лагу.
- Циклічність: Чітко видно періодичну структуру з повторюваними хвилями.  Максимуми та мінімуми чергуються, що вказує на регулярні зміни в генерації електроенергії. Висота максимумів і мінімумів зберігається досить стабільною. Амплітуда автокореляції поступово трохи зменшується, але хвилі залишаються чітко видимими при більших лагах.


**Інтерпретація**

Сильна автокореляція на малих лагах свідчить про те що останні значення мають значний вплив на майбутні.
Періодична структура ACF свідчить про наявність регулярного повторюваного патерну в даних. Чергування максимумі/мінімумів з періодом ~100 вказує на регулярні зміни в генерації електроенергії - цикли по 96 записів на добу. 


### Фазові портрети

#### Аналіз прямої автокореляції

In [ ]:
# Аналіз прямої автокореляції
# Зв'язок між значеннями змінної на різних кроках x(k) та x(k−lag)

lags = [1, 2, 8]
df_numeric = df_num.select_dtypes(include=["number"])
columns = df_numeric.columns

for column in columns:
    fig, axes = plt.subplots(1, len(lags), figsize=(len(lags)*4, 4), sharey=True)
    
    for i, lag in enumerate(lags):
        axes[i].plot(df_num[column][lag:], df_num[column][:-lag], linestyle="-", color="blue", alpha=0.5)
        axes[i].set_title(f"{column} (lag={lag})")
        axes[i].set_xlabel("x(k)")
        axes[i].set_ylabel(f"x(k-{lag})")
        axes[i].grid(True)
    
    plt.tight_layout()
    plt.suptitle("", fontsize=16)
    plt.show()


##### Автокореляція power
- **x(k) і 𝑥(𝑘−1)**
Точки не концентруються навколо чіткої діагональної лінії, а утворюють значну ширину. Це свідчить про слабку кореляцію між сусідніми значеннями x(k) і x(k−1). Такий графік є ознакою високого рівня шуму в даних або значних варіацій між сусідніми значеннями.  Загальна форма графіка містити натяк на тренд або нелінійну залежність.

- **x(k) і 𝑥(𝑘−2)**
Відсутність чіткої структури між x(k) і x(k−2). Точки мають значне розсіювання і покривають мажевсб площину грайфіка. Суттєве ослаблення автокореляції.


#### Аналіз динаміки змін у часовому ряді

In [ ]:
# Аналіз динаміки змін у часовому ряді або нестабільності даних
# Аналіз зміни значень (x(k)−x(k−lag)) відносно значень x(k) для різних лагів

lags = [1, 2, 8]  # Масив лагів

# Обираємо числові колонки
df_numeric = df_num.select_dtypes(include=["number"])

columns = df_numeric.columns
# columns = ['power_actual']

# Проходимо по всіх числових колонках
for column_name in columns:
    # Створюємо фігуру з підграфіками для кожної колонки
    fig, axes = plt.subplots(1, len(lags), figsize=(18, 6), sharey=True)
    
    # Проходимо по всіх лагах
    for i, lag in enumerate(lags):
        # Обчислюємо прирости з урахуванням лагу
        df_diff = df_numeric.diff(lag)
        
        # Будуємо фазовий портрет на відповідному підграфіку
        axes[i].plot(df_numeric[column_name][lag:], df_diff[column_name][lag:], linestyle="-", color="blue", alpha=0.5)
        axes[i].set_title(f"lag = {lag}")
        axes[i].set_xlabel(f"x(k)")
        if i == 0:  # Лише для першого графіка додаємо загальну позначку осі y
            axes[i].set_ylabel(f"x(k) - x(k-{lag})")
        axes[i].grid(True)
    
    # Додаємо заголовок для всієї фігури
    fig.suptitle(f"{column_name}", fontsize=16)
    plt.tight_layout()
    plt.show()

##### Кореляція змін power
- **x(k)−x(k−1) (Lag = 1)**
Точки утворюють діагональну структуру зі значним розсіюванням. Це означає, що сусідні значення змінюються досить плавно, але в межах широкого діапазону. Графік виглядає симетрично щодо осі x(k)−x(k−1)=0, що свідчить про відсутність значного систематичного зміщення у змінах.


### Часові ряди

In [ ]:
df_series = df.copy()

df_series['datetime'] = pd.to_datetime(df_series['datetime'])
df_series.set_index("datetime", inplace=True)

plt.figure(figsize=(15, 20))
cols = ["power_actual", "gti_actual"]
for i, col in enumerate(cols):
    plt.subplot(5, 1, i + 1)
    plt.plot(df_series.index, df_series[col], label=col, alpha=0.7)
    plt.title(f"Time Series of {col}")
    plt.xlabel("Datetime")
    plt.ylabel(col)
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(axis="both", linestyle="--", alpha=0.7)
    plt.tight_layout()

plt.show()

In [ ]:
df_series = df.copy()

df_series['datetime'] = pd.to_datetime(df_series['datetime'])
df_series.set_index("datetime", inplace=True)

plt.figure(figsize=(15, 20))
cols = ["power_actual", "gti_predict", "ghi_predict", "dni_predict"]
for i, col in enumerate(cols):
    plt.subplot(5, 1, i + 1)
    plt.plot(df_series.index, df_series[col], label=col, alpha=0.7)
    plt.title(f"Time Series of {col}")
    plt.xlabel("Datetime")
    plt.ylabel(col)
    plt.legend()
    plt.xticks(rotation=45)
    plt.grid(axis="both", linestyle="--", alpha=0.7)
    plt.tight_layout()

plt.show()

#### datetime

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'])

#### limitation

In [ ]:
# Замінюємо NaN на 0 в колонці "limitation"
df["limitation_actual"] = df["limitation_actual"].fillna(0)

#### power

In [ ]:
# обмеження значень потужності по максимальній потужності станції 
df['power_acual'] = df['power_actual'].clip(lower=0, upper=power_max)

In [ ]:

if drop_invalid_values:
    # видалення невалідних значень
    df = df.dropna(subset=['power_actual'])
else:
    # додати маску для записів, які потрібно виключити з навчання моделі
    df['power_actual'] = df['power_actual'].where((~df['power_actual'].isna()) & (df['limitation_actual'] != 1), -999)

### Класифікація кодів погоди

Коди погоди можна згрупувати за їхнім впливом на сонячну радіацію:
- Ясна погода (мінімальний вплив):
    Коди: 0 (ясне небо)
    Вплив: Максимальна сонячна генерація.

- Хмарність (середній вплив):
    Коди: 1, 2, 3 (різні ступені хмарності)
    Вплив: Залежить від щільності хмарності
    
- Часточки в атмосфері (сильний вплив):
    Коди: 45, 48 (туман, паморозевий туман)
    Вплив: Значне зниження радіації через розсіювання

- Опади (критичний вплив):
    Мряка: 51, 53, 55
    Дощ: 61, 63, 65, 80, 81, 82
    Замерзаючий дощ: 66, 67
    Снігопад: 71, 73, 75, 85, 86
    Снігові зерна: 77
    Грози: 95, 96, 99
    Вплив: Значне зниження генерації через щільні хмари і опади

# Створення моделей

### Обробка даних

In [ ]:
# Завантаження даних з прогнозними меткопараметрами
df_predict = pd.read_csv(f'../Data/{data_name_predict}.csv', sep=',', comment='#', low_memory=False)

# Виключення колонок 'power' та 'limitation', якщо вони є
df_predict = df_predict[[col for col in df_predict.columns if col not in ['power', 'limitation']]]
# Перейменування колонок
df_predict = df_predict.rename(columns={col: f"{col}_predict" for col in df_predict.columns if col != 'datetime'})


# Завантаження даних з фактичними даними аотужності
df_actual = pd.read_csv(f'../Data/{data_name_actual}.csv', sep=',', comment='#', low_memory=False)
df_actual = df_actual.rename(columns={col: f"{col}_actual" for col in df_actual.columns if col != 'datetime'})

df = pd.DataFrame()
df = pd.merge(df_actual, df_predict, on='datetime', how='outer', suffixes=('_actual', '_predict'))

df.columns

In [ ]:
df['datetime'] = pd.to_datetime(df['datetime'], format='mixed', errors='coerce')

In [ ]:
# обмеження значень по максимальній потужності станції 
df['power_acual'] = df['power_actual'].clip(lower=0, upper=power_max)

In [ ]:
# Видалення невалідних записів при обмеження генерації
df["limitation_actual"] = df["limitation_actual"].fillna(0)
df = df[df['limitation_actual'] == 0]
df.drop(columns=['limitation_actual'], inplace=True)

# Видалення невалідних значень потужності
df = df.dropna(subset=['power_actual'])

### Розрахунок метрик

In [ ]:
# Розрахунок метрик
def evaluate_data(_forecast, _actual, _power_max=0, _print_metrics=False):

        # _actual = np.where(_actual == 0, 1e-8, _actual)
    
        if _power_max == 0:
            _power_max = np.max(_actual)

        tae = np.sum(np.abs(_forecast - _actual))

        mae = mean_absolute_error(_actual, _forecast)
        mae_max = (mae / _power_max) * 100

        ape_koef = 0.1
        mask = _actual > ape_koef * _power_max
        if np.any(mask):
            ape_filtered = np.mean(np.abs((_forecast[mask] - _actual[mask]) / _actual[mask])) * 100
        else:
            ape_filtered = np.nan

        mape = mean_absolute_percentage_error(_actual, _forecast) * 100
        mape_max = np.mean(np.abs((_forecast - _actual) / _power_max)) * 100  

        def symmetric_mean_absolute_percentage_error(actual, forecast):
            numerator = np.abs(actual - forecast)
            denominator = (np.abs(actual) + np.abs(forecast)) / 2
            smape = np.mean(numerator / denominator) * 100
            return smape
        smape = symmetric_mean_absolute_percentage_error(_actual, _forecast)

        mse = mean_squared_error(_actual, _forecast)

        rmse = np.sqrt(mse)
        rmse_max = np.sqrt(mse) / _power_max * 100

        r2 = r2_score(_actual, _forecast)
        ev = explained_variance_score(_actual, _forecast)

        bias = np.mean(_forecast - _actual) / _power_max * 100

        msle = mean_squared_log_error(_actual - np.min(_actual) + 1,
                                      _forecast - np.min(_forecast) + 1)
        try:
            pcc, _ = pearsonr(_actual - np.min(_actual) + 1,
                              _forecast - np.min(_forecast) + 1)
        except ValueError: pcc = np.nan

        metrics = {
            "TAE": tae,
            "MAE": mae,
            "MAE(max)": mae_max,
            "MSE": mse,
            "R2": r2,
            "MSLE": msle,
            "Bias": bias,
            "APE": ape_filtered,
            "MAPE": mape,
            "MAPE(max)": mape_max,
            "sMAPE": smape,
            "RMSE": rmse,
            "RMSE(max)": rmse_max,
            "EV": ev,
            "PCC": pcc
        }

        if _print_metrics:
            for key, value in metrics.items():
                print(f"{key}: {value:.3f}")

        return metrics

Метрики прогнозу

- `Generation` - агальний обсяг реальної генерації енергії.
- `Prediction` - загальний обсяг прогнозованої генерації енергії.
- `TAE (Total Absolute Error)` - загальна похибка між реальними та прогнозованими значеннями.
- `APE (Absolute Percentage Error)` - відносна похибка, що показує відхилення прогнозу від реальної генерації у відсотках. `APE = (TAE / Generation) * 100`
- `Bias` - нормалізоване зміщення прогнозу. Пояснює систематичне відхилення між прогнозом і реальністю. `Bias = mean((generation - prediction) / generation) * 100`
`Bias > 0`, модель переоцінює значення (прогноз вище реального), `Bias < 0` - модель має негативне зміщення, недооцінює значення (прогноз нижче реального).
- `MAE (Mean Absolute Error)` - середня абсолютна помилка. Середнє абсолютне відхилення прогнозу від реальних даних. `MAE = mean(|generation - prediction|)`
- `MAPE (Mean Absolute Percentage Error)` - середня абсолютна похибка у відсотках. `MAPE = mean(|(generation - prediction) / generation|) * 100`
- `MAE (mean)` - MAE відносно середньої генерації. `MAE (mean) = (MAE / mean(generation)) * 100`
- `MAE (max)` - MAE відносно максимальної генерації. `MAE (max) = (MAE / max(generation)) * 100`
- `MSE (Mean Squared Error)` - середньоквадратична похибка. `MSE = mean((generation - prediction)^2)`. Чутлива до великих відхилень похибка.
- `RMSE (Root Mean Squared Error)` - корінь середньоквадратичної похибки. `RMSE = sqrt(MSE)`. Пояснює середнє відхилення прогнозу від реальних даних.
- `RMSE (mean)` - RMSE відносно середньої генерації. `RMSE (mean) = (RMSE / mean(generation)) * 100`
- `RMSE (max)` - RMSE відносно максимальної генерації. `RMSE (max) = (RMSE / max(generation)) * 100`
- `R2 (Coefficient of Determination)` - Коефіцієнт детермінації. `R2 = 1 - (sum((generation - prediction)^2) / sum((generation - mean(generation))^2))`. Якість відповідності прогнозу фактичним даним.
- `EV (Explained Variance)` - пояснена дисперсія. `EV = 1 - (variance(generation - prediction) / variance(generation))`. Показує наскільки добре модель пояснює варіації фактичними даних.
- `MSLE (Mean Squared Logarithmic Error)` - середньоквадратична логарифмічна похибка. `MSLE = mean((log(generation + 1) - log(prediction + 1))^2)`. Відносна похибка з урахуванням логарифмічної шкали.
- `PCC (Pearson Correlation Coefficient)` - коефіцієнт кореляції Пірсона. `PCC = cov(generation, prediction) / (std(generation) * std(prediction))`. Лінійна залежність між прогнозом і фактичними даними.


### Відображення даних

In [ ]:
# Відображення кількох часових рядів на одному графіку
def plot_multiple_time_series(df, cols, start_date=None, end_date=None, title="Часовий ряд", figsize=(15, 10), grid=True):

    # Фільтр за діапазоном дат
    if start_date or end_date:
        filtered_df = df.loc[start_date:end_date]
    else:
        filtered_df = df

    plt.figure(figsize=figsize)
    for col in cols:
        plt.plot(filtered_df.index, filtered_df[col], label=col, alpha=0.7)
    
    plt.title(title)
    plt.xlabel("Datetime")
    plt.ylabel("Value")
    plt.legend()
    plt.xticks(rotation=45)
    if grid:
        plt.grid(axis="both", linestyle="--", alpha=0.7)
    plt.tight_layout()
    plt.show()

In [ ]:
def plot_multiple_time_series_split(df, columns=['power_actual', 'prediction_actual'], plot_period='9D', start_date=None, end_date=None):

    if columns is None:
        columns = df.columns.drop('datetime')
        
    # Перетворення datetime у DataFrame до tz-naive (без часового поясу)
    df["datetime"] = pd.to_datetime(df["datetime"]).dt.tz_localize(None)

    # Перевіряємо, чи передані start_date і end_date, і встановлюємо їх за замовчуванням, якщо не передані
    if start_date is None:
        start_date = pd.to_datetime(df["datetime"].min())
    else:
        start_date = pd.to_datetime(start_date).tz_localize(None)

    if end_date is None:
        end_date = pd.to_datetime(df["datetime"].max())
    else:
        end_date = pd.to_datetime(end_date).tz_localize(None)

    # Фільтруємо дані на основі start_date і end_date
    df = df[(df["datetime"] >= start_date) & (df["datetime"] <= end_date)]

    # Якщо plot_period дорівнює '', будуємо один графік для всього діапазону
    if plot_period == '':
        plt.figure(figsize=(30, 6))
        for i, column in enumerate(columns):
            if i == 0:  # Перша колонка
                plt.plot(df["datetime"], df[column], label=column, linewidth=3, color='black')
            else:
                plt.plot(df["datetime"], df[column], label=column)
        
        plt.title(f'Data Visualization: {start_date.date()} - {end_date.date()}')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.show()
        return

    # Розбиваємо на інтервали, якщо plot_period не порожній
    date_ranges = pd.date_range(start=start_date, end=end_date, freq=plot_period)

    # Додаємо останній день (якщо потрібно)
    if date_ranges[-1] < end_date:
        date_ranges = date_ranges.append(pd.DatetimeIndex([end_date]))

    # Виводимо графіки для кожного інтервалу
    for i in range(len(date_ranges) - 1):
        # Фільтруємо дані для поточного інтервалу
        filtered_df = df[
            (df["datetime"] >= date_ranges[i]) & 
            (df["datetime"] < date_ranges[i + 1])
        ]
        
        # Створюємо графік
        plt.figure(figsize=(30, 6))
        for j, column in enumerate(columns):
            if j == 0:  # Перша колонка
                plt.plot(filtered_df["datetime"], filtered_df[column], label=column, linewidth=3, color='black')
            else:
                plt.plot(filtered_df["datetime"], filtered_df[column], label=column)
        
        plt.title(f'Data Visualization: {date_ranges[i].date()} - {date_ranges[i+1].date()}')
        plt.xlabel('Time Steps')
        plt.ylabel('Values')
        plt.legend()
        plt.show()


In [ ]:
def plot_loss(history, title = f'Training and Validation Loss'):
    plt.figure(figsize=(5, 3))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss (MSE)')
    plt.title(title)
    plt.legend()
    plt.grid(True)
    plt.show()

## LSTM 

### Обробка даних

In [ ]:
df.columns
df

### Створення ознак

In [ ]:
# Конвертувати datetime у об'єкт datetime бібліотеки pandas
df["datetime"] = pd.to_datetime(df["datetime"], utc=True)

# Виділити день і годину для трансформацій синуса та косинуса
df["hour"] = df["datetime"].dt.hour
df["day"] = df["datetime"].dt.dayofyear
df["hour_sin"] = np.sin(2 * np.pi * df["hour"] / 24)
df["hour_cos"] = np.cos(2 * np.pi * df["hour"] / 24)
df["day_sin"] = np.sin(2 * np.pi * df["day"] / 365.25)
df["day_cos"] = np.cos(2 * np.pi * df["day"] / 365.25)

In [ ]:
# Розрахунок висоти сонця [apparent_elevation_predict]
def calculate_apparent_elevation(row):
    solar_position = pvlib.solarposition.get_solarposition(
        time=row['datetime'],
        latitude=latitude,
        longitude=longitude,
    )
    return max(solar_position['apparent_elevation'].iloc[0],0)

df['apparent_elevation_predict'] = df.apply(calculate_apparent_elevation, axis=1)
df['is_night_predict'] = df['apparent_elevation_predict'] <= 0

### Підготовка даних для навчання

In [ ]:
features_train = [
       'power_actual',
       
       # 'gti_actual',
       # 'ghi_actual',

       # 'temperature_air_actual',
       # 'temperature_pv_actual',

       'gti_predict',
       'ghi_predict',
       'dni_predict',

       'temperature_air_predict',
       'temperature_soil_predict',

       'cloud_cover_high_predict',
       'cloud_cover_low_predict',
       'cloud_cover_mid_predict',
       
       'dew_point_predict',
       'humidity_predict',
       'pressure_air_predict',
       'sunshine_predict',
       'visibility_predict',
       'wind_speed_predict',
       'wind_direction_predict',
       'weather_code_predict',

       'hour_sin',
       'hour_cos',
       'day_sin',
       'day_cos',
       'apparent_elevation_predict',
       'is_night_predict',
       ]

In [ ]:
# features_train = [
#        'power_actual',
       
#        'toa_predict','clear sky ghi_predict','clear sky bhi_predict','clear sky dhi_predict','clear sky bni_predict','ghi_predict','bhi_predict','dhi_predict','bni_predict',

#        'hour_cos',
#        'apparent_elevation_predict', 'is_night_predict',
#        ]

In [ ]:
df_train = df[features_train].copy()

In [ ]:
plt.figure(figsize=(15, 10))
sns.heatmap(df_train.corr(method="spearman"), annot=True, cmap='coolwarm')
plt.title('Матриця кореляції (spearman)', fontsize=16)
plt.tight_layout()  
plt.show()

In [ ]:
df_train.dropna(inplace=True)

In [ ]:
df_train

In [ ]:
df_train.columns

In [ ]:
# Вибрані ознаки
feature_columns = [col for col in df_train.columns if col != "power_actual"]

# feature_columns = ['gti', 'temperature_air', 'humidity', 'apparent_elevation', 'is_night', 'hour_sin', 'hour_cos', 'day_sin', 'day_cos']
# feature_columns = [col for col in feature_columns if col in df_train.columns]

# Цільова змінна
target_column = 'power_actual'

# Вибірка даних для моделі
# Перша колонка - цільова змінна power, інші - ознаки
df_train = df_train[[target_column] + feature_columns]

In [ ]:
# Розділення даних на навчальну та тестову вибірки
split_ratio = 0.8
sequence_length = 4
forecast_steps = 4

split_index = int(len(df_train) * split_ratio)
data_train = df_train.iloc[:split_index]
data_test = df_train.iloc[split_index:]

In [ ]:
# Масштабування даних для навчання моделей

scaler = MinMaxScaler()
data_train_scaled = scaler.fit_transform(data_train)
data_test_scaled = scaler.transform(data_test)

def prepare_sequences(scaled_data, original_data, sequence_length, forecast_steps):
    X, y, datetimes = [], [], []
    for i in range(sequence_length, len(scaled_data) - forecast_steps + 1):
        y.append(scaled_data[i:i+forecast_steps, 0])    # цільова змінна 'power_actual'
        X.append(scaled_data[i-sequence_length:i, 1:])  # ознаки
        datetimes.append(original_data.index[i+forecast_steps-1])
    return np.array(X), np.array(y), datetimes


X_train, y_train, datetimes_train = prepare_sequences(data_train_scaled, data_train, sequence_length, forecast_steps)
print(f"Length of X_train: {len(X_train)}")
print(f"Length of y_train: {len(y_train)}")
print(f"Length of datetimes_train: {len(datetimes_train)}")

X_test, y_test, datetimes_test = prepare_sequences(data_test_scaled, data_test, sequence_length, forecast_steps)
print(f"Length of X_test: {len(X_test)}")
print(f"Length of y_test: {len(y_test)}")
print(f"Length of datetimes_test: {len(datetimes_test)}")

print(f"X_train size: {len(X_train)}")
print(f"X_test size: {len(X_test)}")
print(f"Features ({len(feature_columns)}): {feature_columns}")

### Моделі

In [ ]:
models_list = []

In [ ]:

model_lstm = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
 
    LSTM(50, return_sequences=False),
    Dense(forecast_steps)
])
model_lstm.compile(optimizer='nadam', loss=['mse'])
models_list.append({
    'name': "lstm",
    'model': model_lstm,
    'params': None,
    'data': None,
    'history': None,
    'metrics': None
})

lstm_extended = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    LSTM(
        units=128,
        activation='relu',
        return_sequences=True,
    ),
    Dropout(0.1),
    BatchNormalization(),
    LSTM(
        units=256,
        activation='relu',
        return_sequences=True,
    ),
    Dropout(0.1),
    BatchNormalization(),
    LSTM(
        units=128,
        activation='relu',
        return_sequences=True,
    ),
    LSTM(50, return_sequences=False),
    Dense(forecast_steps, activation='linear')
])
lstm_extended.compile(optimizer='adam', loss='mse', metrics=['mae', 'msle'])

models_list.append({
    'name': "lstm_extended",
    'model': lstm_extended,
    'params': None,
    'data': None,
    'history': None,
    'metrics': None
})

### Гібридна модель CNN + LSTM

In [ ]:
model_cnn_lstm = Sequential([

    Input(shape=(X_train.shape[1], X_train.shape[2])),
    
    # CNN блок
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),  # Додано padding='same'
    Dropout(0.2),
    
    # Другий CNN блок
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),
    Dropout(0.2),
    
    Reshape((-1, 64)),  # -1 визначає розмір автоматично, 64 — кількість фільтрів з CNN
    
    # LSTM блок
    LSTM(50, return_sequences=True),
    Dropout(0.1),
    LSTM(50, return_sequences=True),
    Dropout(0.1),
    LSTM(50, return_sequences=False),
    Dropout(0.1),
    
    # Fully Connected Layers
    Dense(25, activation='relu'),
    Dense(forecast_steps)
])

# Компіляція моделі
model_cnn_lstm.compile(optimizer='adam', loss='mse')

models_list.append({
    'name': "cnn_lstm",
    'model': model_cnn_lstm,
    'params': None,
    'data': None,
    'history': None,
    'metrics': None
})


### Гібридна модела CNN + GRU

In [ ]:
model_cnn_gru = Sequential([
    Input(shape=(X_train.shape[1], X_train.shape[2])),
    
    # CNN блок
    Conv1D(filters=64, kernel_size=3, activation='relu', padding='same'),  # Додано padding='same'
    MaxPooling1D(pool_size=2, padding='same'),  # Щоб уникнути зменшення розміру до негативного
    GRU(50, return_sequences=True),
    Dropout(0.1),
    GRU(50, return_sequences=False),
    Dense(forecast_steps)
])

# Компіляція моделі
model_cnn_gru.compile(optimizer='adam', loss='mse')

models_list.append({
    'name': "cnn_gru",
    'model': model_cnn_gru,
    'params': None,
    'data': None,
    'history': None,
    'metrics': None
})


### Гібридна модела CNN + Transformer

In [ ]:
# Побудова гібридної моделі CNN + Transformer

# Вхідний шар
inputs = Input(shape=(X_train.shape[1], X_train.shape[2]))

# CNN частина
x = Conv1D(filters=64, kernel_size=3, activation='relu')(inputs)
x = MaxPooling1D(pool_size=2)(x)

# Transformer частина
# Для MultiHeadAttention потрібні query, key і value
attn_output = MultiHeadAttention(num_heads=4, key_dim=64)(x, x)
x = Flatten()(attn_output)

# Повнозв'язний шар
x = Dense(25, activation='relu')(x)
outputs = Dense(1)(x)

# Модель
model_cnn_tranformers = Model(inputs=inputs, outputs=outputs)

# Компіляція моделі
model_cnn_tranformers.compile(optimizer='adam', loss='mse')

models_list.append({
    'name': "cnn_tranformers",
    'model': model_cnn_tranformers,
    'params': None,
    'data': None,
    'history': None,
    'metrics': None
})

### Навчання моделей, прогнозування

In [ ]:
for model_ in models_list:

    model_name = model_['name']
    print(f"==> {model_name}")
    
    model_object = model_['model']

    # Тренування моделі
    model_['history'] = model_object.fit(
        X_train, y_train,
        validation_data=(X_test, y_test),
        epochs=32,
        batch_size=32,
        callbacks=[EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)],
        verbose=1)
    
    # Отримання прогнозованих значень
    predictions = model_object.predict(X_test)

    predictions_rescaled = []
    y_test_rescaled = []

    # Відновлення масштабу для тестових даних
    n_features = scaler.n_features_in_    
    y_test_rescaled = scaler.inverse_transform(
        np.c_[y_test[:, 0], np.zeros((y_test.shape[0], n_features - 1))]
    )[:, 0]

    # Відновлення масштабу для прогнозу
    forecast_rescaled = scaler.inverse_transform(
        np.c_[predictions[:, 0], np.zeros((predictions.shape[0], n_features - 1))]
    )[:, 0]

    # Створення DataFrame з відновленими даними
    results_df = pd.DataFrame({
        "datetime": datetimes_test,
        "power_actual": y_test_rescaled,
        "power_predict": forecast_rescaled
    })

    model_['data'] = results_df.copy()

    # Розрахунок метрик
    model_['metrics'] = evaluate_data(model_['data']['power_actual'], model_['data']['power_predict'], _power_max=power_max, _print_metrics=True)

### Криві навчання

In [ ]:
for model_object in models_list:
    plot_loss(model_object['history'], title = f'Training and Validation Loss for {model_object['name']}')

### Оцінка метрик

In [ ]:
for model in models_list:
    model['metrics']['Модель'] = model.get('name')

# Вибір колонок для виводу
selected_columns = ["Модель", "APE", "MAE(max)", "RMSE(max)", "R2", "MSLE", "Bias", "EV", "PCC"]

# Формування даних для таблиці
metrics_array = [
    {key: model['metrics'][key] for key in selected_columns if key in model['metrics']}
    for model in models_list
]

# Створення таблиці та округлення
metrics_table = pd.DataFrame(metrics_array)
metrics_table = metrics_table.round(2)
metrics_table.set_index('Модель', inplace=True)
metrics_table['Рейтиг'] = metrics_table[['APE', 'MAE(max)', 'RMSE(max)']].sum(axis=1).rank(ascending=True)

# Відображення таблиці
display(metrics_table.sort_values(by='Рейтиг'))

### Часові ряди

In [ ]:
# Відображення часових рядів
def plot_series(_data, _columns, _title="Фактична і Прогнозована потужність", _metrics=None, _split_days=0):

    # Дата-час може бути в індексі або в колонці "datetime"
    if "datetime" in _data.columns:
        datetime_col = _data["datetime"]
    else:
        datetime_col = _data.index

    start_index = 0
    end_index = len(_data)
    data_plot = _data.iloc[start_index:end_index].copy()
    datetime_col_plot = datetime_col[start_index:end_index]

    # Візуалізація графіків
    plt.figure(figsize=(15, 5))
    for col in _columns:
        plt.plot(datetime_col_plot, data_plot[col["column"]], label=col["label"])
    plt.title(f"{_title}")
    
    # Візуалізація метрик
    metrics_text = ""
    if _metrics is not None:
        metrics_text = "\n".join([
            f'MAE(max) = {_metrics["MAE(max)"]:.1f}%',
            f'MAPE(max) = {_metrics["MAPE(max)"]:.1f}%',
            f'MAPE(max) = {_metrics["MAPE(max)"]:.1f}%',            
            f'R2 = {_metrics["R2"]:.3f}',
            f'MSLE = {_metrics["MSLE"]:.3f}',
            f'Bias = {_metrics["Bias"]:.1f}',
            # f'APE = {_metrics["APE"]:.2f}%',
            # f'sMAPE = {_metrics["MAPE"]:.1e}',
            # f'EV = {_metrics["EV"]:.3f}',
            # f'PCC = {_metrics["PCC"]:.3f}'
        ])
        
    plt.legend(loc='upper left', title=metrics_text)
    
    plt.grid(True)
    plt.show()

    # Візуалізація графіків по частинах
    if _split_days > 0:
        data_len = len(_data)
        days_in_step = _split_days * 24 * 4
        for i in range(0, data_len, days_in_step):
            data_chunk = _data.iloc[i:i + days_in_step]
            datetime_chunk = datetime_col.iloc[i:i + days_in_step]

            plt.figure(figsize=(15, 5))
            for col in _columns:
                plt.plot(datetime_chunk, data_chunk[col["column"]], label=col["label"])
            plt.title(f"{_title} - Інтервал {i // days_in_step + 1}")
            plt.legend(loc='upper left')
            plt.grid(True)
            plt.show()


In [ ]:
for model_ in models_list:
  
    # Оцінка прогнозу і візуалізація часовових рядів потужності
    print(f"\n\n==> {model_['name']}\n")
    plot_series(_data=model_['data'],
                _columns=  [{"column": "power_actual", "label": "Факт", "power_max": 0},
                            {"column": "power_predict", "label": "Прогноз", "power_max": 0}],
                _title=f"Фактична і Прогнозована потужність ({model_['name']})",
                _metrics=model_['metrics'],
                )
   